In [17]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [18]:
#定义变量 
agent_resource_quantity = np.zeros(shape=(54,2000), dtype=np.float64) #100个agent的资源量 定义成一个矩阵
sum_resource=np.zeros(shape=(1,2000), dtype=np.float64)#存放每一次迭代后所有用户资源量的总和
sum_cost=np.zeros(shape=(1,2000), dtype=np.float64)#存放每一次迭代后所有用户成本的总和
iteration_update = 1 #迭代步长 
A=np.loadtxt("Adjacency_Matrix_generator_zero.txt") #网络拓扑的邻接矩阵 对角元素要为0
D=np.zeros(shape=(54,54))# 网络的度矩阵
L=np.zeros(shape=(54,54))# 网络的拉普拉斯矩阵 L=D-A
W=np.zeros(shape=(54,54)) #可以直接采用网络对应的拉普拉斯矩阵  因为无向图的拉普拉斯矩阵天然满足行和和列和为0
iteration=[]#绘图横坐标
generator_a_values= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
agent_gradient=np.zeros(shape=(54,2000), dtype=np.float64) 
bias=80

In [19]:
#根据邻接矩阵生成网络对应的度矩阵
for i in range(0,54):
    d=0
    for j in range(0,54):
        if A[i][j]==1:
            d=d+1
    D[i][i]=d

#根据邻接矩阵和度矩阵计算拉普拉斯矩阵
L=D-A

#选择用拉普拉斯矩阵作为聚合矩阵
W=L
    
#资源量的初始化 需要给定可行的初始化 即满足平均资源约束
for i in range (0,54):
    agent_resource_quantity[i][0]=6000/54
    
#总资源量 初始化
sum=0
for i in range (0,54):
    sum=sum+agent_resource_quantity[i][0]
sum_resource[0]=sum

iteration.append(0)

#初始梯度计算 随机选择一个用户作为biased 用户
for i in range (0,54):
    if i==48:
        agent_gradient[i][0]=2*generator_c_values[i]*agent_resource_quantity[i][0]+generator_b_values[i]+bias
    else:
        agent_gradient[i][0]=2*generator_c_values[i]*agent_resource_quantity[i][0]+generator_b_values[i]
        

#用户总成本初始化
sum=0
for i in range(0,54):
    sum=sum+(generator_c_values[i]*(agent_resource_quantity[i][0])*(agent_resource_quantity[i][0])+generator_b_values[i]*agent_resource_quantity[i][0]+generator_a_values[i])
sum_cost[0][0]=sum

In [20]:
#基于初始化的权重梯度下降无中心资源分配算法
def IWDGD_deterministic_bias(generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************IWDGD算法+计算误差****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量总和为:",sum_resource[0][0])
    print("*************************")
    print("初始成本总和为:",sum_cost[0][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,2000):#迭代次数
        iteration.append(k)
 
        #所有用户资源申请量基于可行初始化权重梯度下降的迭代  假设部分用户因为设备损坏 导致梯度的计算有误差     
        for i in range (0,54):
            weight_gradient_sum = np.float64(0)
            for j in range (0,54):        
                weight_gradient_sum=weight_gradient_sum+W[i][j]*agent_gradient[j][k-1]
            #weight_gradient_sum = np.round(weight_gradient_sum, decimals=3)
            agent_resource_quantity[i][k]=agent_resource_quantity[i][k-1]-iteration_update*weight_gradient_sum
                #agent_resource_quantity[i][k] = np.round(agent_resource_quantity[i][k], decimals=3)
        
        #计算所有用户的资源量总和
        sum1=np.float64(0)
        for i in range (0,54):
            sum1 = sum1 + agent_resource_quantity[i][k]
        sum_resource[0][k]=sum1
        
        #计算所有用户的成本总和
        sum2=np.float64(0)
        for i in range(0,54):
            sum2=sum2+(generator_c_values[i]*(agent_resource_quantity[i][k])*(agent_resource_quantity[i][k])+generator_b_values[i]*agent_resource_quantity[i][k]+generator_a_values[i])
        sum_cost[0][k]=sum2
        
        #计算当前迭代的用户梯度
        for i in range (0,54): 
            if i==48:
                agent_gradient[i][k]=2*generator_c_values[i]*agent_resource_quantity[i][k]+generator_b_values[i]+bias
            else:
                agent_gradient[i][k]=2*generator_c_values[i]*agent_resource_quantity[i][k]+generator_b_values[i]
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%20==0:
        #if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,54):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
            print("资源申请量总和为:",sum_resource[0][k])
            print("****************************************************************************")
            print("成本总和为:",sum_cost[0][k])
            print("****************************************************************************")
            print("迭代步长为:",iteration_update)
            print("****************************************************************************")
        
        #iteration_update=1/(pow(k+1,0.5)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [21]:
IWDGD_deterministic_bias(generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias)

****************************IWDGD算法+计算误差****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 111.11111111111111
agent 2 的初始资源申请量为: 111.11111111111111
agent 3 的初始资源申请量为: 111.11111111111111
agent 4 的初始资源申请量为: 111.11111111111111
agent 5 的初始资源申请量为: 111.11111111111111
agent 6 的初始资源申请量为: 111.11111111111111
agent 7 的初始资源申请量为: 111.11111111111111
agent 8 的初始资源申请量为: 111.11111111111111
agent 9 的初始资源申请量为: 111.11111111111111
agent 10 的初始资源申请量为: 111.11111111111111
agent 11 的初始资源申请量为: 111.11111111111111
agent 12 的初始资源申请量为: 111.11111111111111
agent 13 的初始资源申请量为: 111.11111111111111
agent 14 的初始资源申请量为: 111.11111111111111
agent 15 的初始资源申请量为: 111.11111111111111
agent 16 的初始资源申请量为: 111.11111111111111
agent 17 的初始资源申请量为: 111.11111111111111
agent 18 的初始资源申请量为: 111.11111111111111
agent 19 的初始资源申请量为: 111.11111111111111
agent 20 的初始资源申请量为: 111.11111111111111
agent 21 的初始资源申请量为: 111.11111111111111
agent 22 的初始资源申请量为: 111.11111111111111
agent 23 的初始资源申请量为: 111.11111111111111
agent 24 的初始资源申请量为: 111.111111111

59 次迭代中的结果：
agent 1 在 59 次迭代中的资源申请量为: 52.91701066297617
agent 2 在 59 次迭代中的资源申请量为: 66.62126332872022
agent 3 在 59 次迭代中的资源申请量为: 47.79147999674369
agent 4 在 59 次迭代中的资源申请量为: 70.6413781767857
agent 5 在 59 次迭代中的资源申请量为: 71.4413781767857
agent 6 在 59 次迭代中的资源申请量为: 78.82551599446427
agent 7 在 59 次迭代中的资源申请量为: 43.03448210234963
agent 8 在 59 次迭代中的资源申请量为: 41.11275799723215
agent 9 在 59 次迭代中的资源申请量为: 47.09147999674371
agent 10 在 59 次迭代中的资源申请量为: 60.99655076497254
agent 11 在 59 次迭代中的资源申请量为: 49.71594749654019
agent 12 在 59 次迭代中的资源申请量为: 53.817010662976216
agent 13 在 59 次迭代中的资源申请量为: 71.94137817678575
agent 14 在 59 次迭代中的资源申请量为: 60.39655076497257
agent 15 在 59 次迭代中的资源申请量为: 47.49147999674373
agent 16 在 59 次迭代中的资源申请量为: 47.69147999674374
agent 17 在 59 次迭代中的资源申请量为: 49.11594749654022
agent 18 在 59 次迭代中的资源申请量为: 50.71594749654022
agent 19 在 59 次迭代中的资源申请量为: 45.08084221914686
agent 20 在 59 次迭代中的资源申请量为: 60.89655076497258
agent 21 在 59 次迭代中的资源申请量为: 42.33448210234966
agent 22 在 59 次迭代中的资源申请量为: 42.634482102349665
agent 2

109 次迭代中的结果：
agent 1 在 109 次迭代中的资源申请量为: 52.91701066297617
agent 2 在 109 次迭代中的资源申请量为: 66.62126332872022
agent 3 在 109 次迭代中的资源申请量为: 47.79147999674369
agent 4 在 109 次迭代中的资源申请量为: 70.6413781767857
agent 5 在 109 次迭代中的资源申请量为: 71.4413781767857
agent 6 在 109 次迭代中的资源申请量为: 78.82551599446427
agent 7 在 109 次迭代中的资源申请量为: 43.03448210234963
agent 8 在 109 次迭代中的资源申请量为: 41.11275799723215
agent 9 在 109 次迭代中的资源申请量为: 47.09147999674371
agent 10 在 109 次迭代中的资源申请量为: 60.99655076497254
agent 11 在 109 次迭代中的资源申请量为: 49.71594749654019
agent 12 在 109 次迭代中的资源申请量为: 53.817010662976216
agent 13 在 109 次迭代中的资源申请量为: 71.94137817678575
agent 14 在 109 次迭代中的资源申请量为: 60.39655076497257
agent 15 在 109 次迭代中的资源申请量为: 47.49147999674373
agent 16 在 109 次迭代中的资源申请量为: 47.69147999674374
agent 17 在 109 次迭代中的资源申请量为: 49.11594749654022
agent 18 在 109 次迭代中的资源申请量为: 50.71594749654022
agent 19 在 109 次迭代中的资源申请量为: 45.08084221914686
agent 20 在 109 次迭代中的资源申请量为: 60.89655076497258
agent 21 在 109 次迭代中的资源申请量为: 42.33448210234966
agent 22 在 109 次迭代中的资源申请量为: 42.

169 次迭代中的结果：
agent 1 在 169 次迭代中的资源申请量为: 52.91701066297617
agent 2 在 169 次迭代中的资源申请量为: 66.62126332872022
agent 3 在 169 次迭代中的资源申请量为: 47.79147999674369
agent 4 在 169 次迭代中的资源申请量为: 70.6413781767857
agent 5 在 169 次迭代中的资源申请量为: 71.4413781767857
agent 6 在 169 次迭代中的资源申请量为: 78.82551599446427
agent 7 在 169 次迭代中的资源申请量为: 43.03448210234963
agent 8 在 169 次迭代中的资源申请量为: 41.11275799723215
agent 9 在 169 次迭代中的资源申请量为: 47.09147999674371
agent 10 在 169 次迭代中的资源申请量为: 60.99655076497254
agent 11 在 169 次迭代中的资源申请量为: 49.71594749654019
agent 12 在 169 次迭代中的资源申请量为: 53.817010662976216
agent 13 在 169 次迭代中的资源申请量为: 71.94137817678575
agent 14 在 169 次迭代中的资源申请量为: 60.39655076497257
agent 15 在 169 次迭代中的资源申请量为: 47.49147999674373
agent 16 在 169 次迭代中的资源申请量为: 47.69147999674374
agent 17 在 169 次迭代中的资源申请量为: 49.11594749654022
agent 18 在 169 次迭代中的资源申请量为: 50.71594749654022
agent 19 在 169 次迭代中的资源申请量为: 45.08084221914686
agent 20 在 169 次迭代中的资源申请量为: 60.89655076497258
agent 21 在 169 次迭代中的资源申请量为: 42.33448210234966
agent 22 在 169 次迭代中的资源申请量为: 42.

219 次迭代中的结果：
agent 1 在 219 次迭代中的资源申请量为: 52.91701066297617
agent 2 在 219 次迭代中的资源申请量为: 66.62126332872022
agent 3 在 219 次迭代中的资源申请量为: 47.79147999674369
agent 4 在 219 次迭代中的资源申请量为: 70.6413781767857
agent 5 在 219 次迭代中的资源申请量为: 71.4413781767857
agent 6 在 219 次迭代中的资源申请量为: 78.82551599446427
agent 7 在 219 次迭代中的资源申请量为: 43.03448210234963
agent 8 在 219 次迭代中的资源申请量为: 41.11275799723215
agent 9 在 219 次迭代中的资源申请量为: 47.09147999674371
agent 10 在 219 次迭代中的资源申请量为: 60.99655076497254
agent 11 在 219 次迭代中的资源申请量为: 49.71594749654019
agent 12 在 219 次迭代中的资源申请量为: 53.817010662976216
agent 13 在 219 次迭代中的资源申请量为: 71.94137817678575
agent 14 在 219 次迭代中的资源申请量为: 60.39655076497257
agent 15 在 219 次迭代中的资源申请量为: 47.49147999674373
agent 16 在 219 次迭代中的资源申请量为: 47.69147999674374
agent 17 在 219 次迭代中的资源申请量为: 49.11594749654022
agent 18 在 219 次迭代中的资源申请量为: 50.71594749654022
agent 19 在 219 次迭代中的资源申请量为: 45.08084221914686
agent 20 在 219 次迭代中的资源申请量为: 60.89655076497258
agent 21 在 219 次迭代中的资源申请量为: 42.33448210234966
agent 22 在 219 次迭代中的资源申请量为: 42.

...

1749 次迭代中的结果：
agent 1 在 1749 次迭代中的资源申请量为: 52.91701066297617
agent 2 在 1749 次迭代中的资源申请量为: 66.62126332872022
agent 3 在 1749 次迭代中的资源申请量为: 47.79147999674369
agent 4 在 1749 次迭代中的资源申请量为: 70.6413781767857
agent 5 在 1749 次迭代中的资源申请量为: 71.4413781767857
agent 6 在 1749 次迭代中的资源申请量为: 78.82551599446427
agent 7 在 1749 次迭代中的资源申请量为: 43.03448210234963
agent 8 在 1749 次迭代中的资源申请量为: 41.11275799723215
agent 9 在 1749 次迭代中的资源申请量为: 47.09147999674371
agent 10 在 1749 次迭代中的资源申请量为: 60.99655076497254
agent 11 在 1749 次迭代中的资源申请量为: 49.71594749654019
agent 12 在 1749 次迭代中的资源申请量为: 53.817010662976216
agent 13 在 1749 次迭代中的资源申请量为: 71.94137817678575
agent 14 在 1749 次迭代中的资源申请量为: 60.39655076497257
agent 15 在 1749 次迭代中的资源申请量为: 47.49147999674373
agent 16 在 1749 次迭代中的资源申请量为: 47.69147999674374
agent 17 在 1749 次迭代中的资源申请量为: 49.11594749654022
agent 18 在 1749 次迭代中的资源申请量为: 50.71594749654022
agent 19 在 1749 次迭代中的资源申请量为: 45.08084221914686
agent 20 在 1749 次迭代中的资源申请量为: 60.89655076497258
agent 21 在 1749 次迭代中的资源申请量为: 42.33448210234966
agent 22 

1799 次迭代中的结果：
agent 1 在 1799 次迭代中的资源申请量为: 52.91701066297617
agent 2 在 1799 次迭代中的资源申请量为: 66.62126332872022
agent 3 在 1799 次迭代中的资源申请量为: 47.79147999674369
agent 4 在 1799 次迭代中的资源申请量为: 70.6413781767857
agent 5 在 1799 次迭代中的资源申请量为: 71.4413781767857
agent 6 在 1799 次迭代中的资源申请量为: 78.82551599446427
agent 7 在 1799 次迭代中的资源申请量为: 43.03448210234963
agent 8 在 1799 次迭代中的资源申请量为: 41.11275799723215
agent 9 在 1799 次迭代中的资源申请量为: 47.09147999674371
agent 10 在 1799 次迭代中的资源申请量为: 60.99655076497254
agent 11 在 1799 次迭代中的资源申请量为: 49.71594749654019
agent 12 在 1799 次迭代中的资源申请量为: 53.817010662976216
agent 13 在 1799 次迭代中的资源申请量为: 71.94137817678575
agent 14 在 1799 次迭代中的资源申请量为: 60.39655076497257
agent 15 在 1799 次迭代中的资源申请量为: 47.49147999674373
agent 16 在 1799 次迭代中的资源申请量为: 47.69147999674374
agent 17 在 1799 次迭代中的资源申请量为: 49.11594749654022
agent 18 在 1799 次迭代中的资源申请量为: 50.71594749654022
agent 19 在 1799 次迭代中的资源申请量为: 45.08084221914686
agent 20 在 1799 次迭代中的资源申请量为: 60.89655076497258
agent 21 在 1799 次迭代中的资源申请量为: 42.33448210234966
agent 22 

1859 次迭代中的结果：
agent 1 在 1859 次迭代中的资源申请量为: 52.91701066297617
agent 2 在 1859 次迭代中的资源申请量为: 66.62126332872022
agent 3 在 1859 次迭代中的资源申请量为: 47.79147999674369
agent 4 在 1859 次迭代中的资源申请量为: 70.6413781767857
agent 5 在 1859 次迭代中的资源申请量为: 71.4413781767857
agent 6 在 1859 次迭代中的资源申请量为: 78.82551599446427
agent 7 在 1859 次迭代中的资源申请量为: 43.03448210234963
agent 8 在 1859 次迭代中的资源申请量为: 41.11275799723215
agent 9 在 1859 次迭代中的资源申请量为: 47.09147999674371
agent 10 在 1859 次迭代中的资源申请量为: 60.99655076497254
agent 11 在 1859 次迭代中的资源申请量为: 49.71594749654019
agent 12 在 1859 次迭代中的资源申请量为: 53.817010662976216
agent 13 在 1859 次迭代中的资源申请量为: 71.94137817678575
agent 14 在 1859 次迭代中的资源申请量为: 60.39655076497257
agent 15 在 1859 次迭代中的资源申请量为: 47.49147999674373
agent 16 在 1859 次迭代中的资源申请量为: 47.69147999674374
agent 17 在 1859 次迭代中的资源申请量为: 49.11594749654022
agent 18 在 1859 次迭代中的资源申请量为: 50.71594749654022
agent 19 在 1859 次迭代中的资源申请量为: 45.08084221914686
agent 20 在 1859 次迭代中的资源申请量为: 60.89655076497258
agent 21 在 1859 次迭代中的资源申请量为: 42.33448210234966
agent 22 

1909 次迭代中的结果：
agent 1 在 1909 次迭代中的资源申请量为: 52.91701066297617
agent 2 在 1909 次迭代中的资源申请量为: 66.62126332872022
agent 3 在 1909 次迭代中的资源申请量为: 47.79147999674369
agent 4 在 1909 次迭代中的资源申请量为: 70.6413781767857
agent 5 在 1909 次迭代中的资源申请量为: 71.4413781767857
agent 6 在 1909 次迭代中的资源申请量为: 78.82551599446427
agent 7 在 1909 次迭代中的资源申请量为: 43.03448210234963
agent 8 在 1909 次迭代中的资源申请量为: 41.11275799723215
agent 9 在 1909 次迭代中的资源申请量为: 47.09147999674371
agent 10 在 1909 次迭代中的资源申请量为: 60.99655076497254
agent 11 在 1909 次迭代中的资源申请量为: 49.71594749654019
agent 12 在 1909 次迭代中的资源申请量为: 53.817010662976216
agent 13 在 1909 次迭代中的资源申请量为: 71.94137817678575
agent 14 在 1909 次迭代中的资源申请量为: 60.39655076497257
agent 15 在 1909 次迭代中的资源申请量为: 47.49147999674373
agent 16 在 1909 次迭代中的资源申请量为: 47.69147999674374
agent 17 在 1909 次迭代中的资源申请量为: 49.11594749654022
agent 18 在 1909 次迭代中的资源申请量为: 50.71594749654022
agent 19 在 1909 次迭代中的资源申请量为: 45.08084221914686
agent 20 在 1909 次迭代中的资源申请量为: 60.89655076497258
agent 21 在 1909 次迭代中的资源申请量为: 42.33448210234966
agent 22 

1969 次迭代中的结果：
agent 1 在 1969 次迭代中的资源申请量为: 52.91701066297617
agent 2 在 1969 次迭代中的资源申请量为: 66.62126332872022
agent 3 在 1969 次迭代中的资源申请量为: 47.79147999674369
agent 4 在 1969 次迭代中的资源申请量为: 70.6413781767857
agent 5 在 1969 次迭代中的资源申请量为: 71.4413781767857
agent 6 在 1969 次迭代中的资源申请量为: 78.82551599446427
agent 7 在 1969 次迭代中的资源申请量为: 43.03448210234963
agent 8 在 1969 次迭代中的资源申请量为: 41.11275799723215
agent 9 在 1969 次迭代中的资源申请量为: 47.09147999674371
agent 10 在 1969 次迭代中的资源申请量为: 60.99655076497254
agent 11 在 1969 次迭代中的资源申请量为: 49.71594749654019
agent 12 在 1969 次迭代中的资源申请量为: 53.817010662976216
agent 13 在 1969 次迭代中的资源申请量为: 71.94137817678575
agent 14 在 1969 次迭代中的资源申请量为: 60.39655076497257
agent 15 在 1969 次迭代中的资源申请量为: 47.49147999674373
agent 16 在 1969 次迭代中的资源申请量为: 47.69147999674374
agent 17 在 1969 次迭代中的资源申请量为: 49.11594749654022
agent 18 在 1969 次迭代中的资源申请量为: 50.71594749654022
agent 19 在 1969 次迭代中的资源申请量为: 45.08084221914686
agent 20 在 1969 次迭代中的资源申请量为: 60.89655076497258
agent 21 在 1969 次迭代中的资源申请量为: 42.33448210234966
agent 22 

In [22]:
np.savetxt('../data/01-agent_resource_quantity_small_value_deterministic_bias.txt',agent_resource_quantity,fmt='%0.8f')#用户资源量
np.savetxt('../data/01-sum_cost_small_value_deterministic_bias.txt',sum_cost,fmt='%0.8f')#用户成本总和
np.savetxt('../data/01-sum_resource_small_value_deterministic_bias.txt',sum_resource,fmt='%0.8f')#用户资源总和
np.savetxt('../data/01-agent_gradient_small_value_deterministic_bias.txt',agent_gradient,fmt='%0.8f')#用户迭代梯度

In [23]:
agent_gradient[0][1999]

np.float64(18.571404520459655)

In [24]:
agent_gradient[10][1999]

np.float64(18.571322975058756)